In [91]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.wait import WebDriverWait
from time import sleep
from selenium.webdriver.support import expected_conditions as EC

from Driver import Driver
from html2image import Html2Image
from util import json_write, json_read
import os, re
import shutil

In [92]:
def save_svg_image(code, img_name, path, size=(250, 250)):
    try: 
        if not os.path.exists(path):
            os.makedirs(path)
    except Exception as bug:
        # print('Create folder EROOR', bug)
        return
    
    list_dir_file = os.listdir(path)

    if img_name in list_dir_file:
        os.remove(f"{path}\\{img_name}")
    
    html_to_image = Html2Image()
    html_to_image.screenshot(html_str=code, save_as=img_name, size=size)
    sleep(0.25)
    shutil.move(img_name, path)
    sleep(0.25)

In [93]:
the_driver = Driver()
driver = the_driver.init_driver(opt='mazzi')
driver.get(kanji_mazzi_domain)

In [94]:
def get_word_data(driver, word):
    result_data = {}
    save_bug = {}

    for x in range(3):
        try:
            # ================= CLOSE BTN CLICK IF HAVE WORD ELSE PASS ====================
            try:
                close_btn_ele = driver.find_element(By.CSS_SELECTOR, 'button[id="clear-search-text"]')
                if close_btn_ele:
                    close_btn_ele.click()
                    sleep(0.5)
            except Exception as bug:
                save_bug.update({'CLOSE BTN': bug})
                pass

            # ================= SEND INPUT VALUE ====================
            input_ele = driver.find_element(By.CSS_SELECTOR, 'input[id="search-text-box"]')
            input_ele.send_keys(word)
            sleep(0.25)

            # ================= SEARCH BTN CLICK ====================
            search_btn_ele = driver.find_element(By.CSS_SELECTOR, 'button[id="search-button"]')
            search_btn_ele.click()
            sleep(0.5)

            # ================= VN KANJI TAB CLICK ====================
            try:
                kanji_tab_ele = driver.find_element(By.CSS_SELECTOR, 'button[class="btn btn-tab-search bg-content-df search-option-kanji"]')
                if kanji_tab_ele:
                    kanji_tab_ele.click()
                    sleep(0.5)
            except Exception as bug:
                kanji_tab_ele_active = driver.find_element(By.CSS_SELECTOR, 'button[class="btn btn-tab-search bg-content-df search-option-kanji tab-active"]')
                if kanji_tab_ele_active:
                    # print('KANJI TAB WAS CLICKED')
                    save_bug.update({'KANJI TAB': 'KANJI TAB WAS CLICKED'})
                pass
            
            # ================= STROKE ====================
            try:
                stroke_ele = driver.find_element(By.CSS_SELECTOR, 'div[class="pronoun-item"]')
                if stroke_ele:
                    stroke_num = int(re.findall(r'(\d+)', stroke_ele.text)[0])
                    print(">> Stroke: ", stroke_num)
                else: stroke_num = 15
            except Exception as bug:
                stroke_num = 2
                save_bug.update({'STROKE': bug})
                print("bug: STROKE")
                pass

            wait_draw_time = 4
            if stroke_num > 4: 
                wait_draw_time = int(stroke_num * 1.5)
            if stroke_num > 10:
                wait_draw_time = int(stroke_num * 1.25)

            for i in range(wait_draw_time):
                # key = ['|', '/', '\\', '/', '\\']
                key_animate = [
                    "[==         ]", "[ ==        ]", "[  ==       ]", "[   ==      ]", "[    ==     ]", "[     ==    ]", "[      ==   ]", "[       ==  ]",
                    "[        == ]", "[         ==]", "[         ==]", "[        == ]", "[       ==  ]", "[      ==   ]", "[     ==    ]", "[    ==     ]",
                    "[   ==      ]", "[  ==       ]", "[ ==        ]", "[==         ]"
                ]
                for j in range(len(key_animate)):
                    print('\r' + f">> Wait draw time: {key_animate[j]} [{wait_draw_time-i:>3}s ]", end='', flush=True)
                    sleep(0.05)
            print('\r' + f">> Wait draw time: {key_animate[len(key_animate)-1]} [ {'OKE':>3} ]", end='', flush=True)
            print('')

            # ================= VN SOUND ====================
            try:
                vn_sound_ele = driver.find_element(By.CSS_SELECTOR, 'h2[class="pronoun-item"] span span')
                if close_btn_ele:
                    vn_sound = vn_sound_ele.text
                    vn_sound = vn_sound.split('-')[1].strip()
                    sleep(0.25)
                else: vn_sound = ''
            except Exception as bug:
                vn_sound = ''
                save_bug.update({'VN SOUND': bug})
                pass
            print(">> Vn sound: ", vn_sound)    
            
            # ================= ONYOMI ====================
            try:
                onyomi_ele = driver.find_element(By.CSS_SELECTOR, 'div[class="col-md-6 col-sm-12 kanji-main-infor"] div:nth-child(2) span:nth-child(2)')
                if onyomi_ele:
                    onyomi = onyomi_ele.text.replace('    ', '')
                    onyomi = [text.strip() for text in onyomi.split('-')]
                    onyomi = '###'.join(onyomi)
                    sleep(0.25)
                else: onyomi = ''
            except Exception as bug:
                onyomi = ''
                save_bug.update({'ONYOMI': bug})
                pass
            print(">> Onyomi: ", onyomi)

            # ================= KUNYOMI ====================
            try:
                kunyomi_ele = driver.find_element(By.CSS_SELECTOR, 'div[class="col-md-6 col-sm-12 kanji-main-infor"] div:nth-child(3) span:nth-child(2)')
                if kunyomi_ele: 
                    kunyomi = kunyomi_ele.text.strip()
                    kunyomi = kunyomi_ele.text
                    kunyomi = [text.strip() for text in kunyomi.split('    ')]
                    kunyomi = '###'.join(kunyomi)
                    sleep(0.25)
                else: kunyomi = ''
            except Exception as bug:
                kunyomi = ''
                save_bug.update({'KUNYOMI': bug})
                pass
            print(">> Kunyomi: ", kunyomi)

            # ================= WORD MEAN ====================
            try:
                word_mean_ele = driver.find_element(By.CSS_SELECTOR, 'div[class="short-mean"]')
                if word_mean_ele: 
                    word_mean = word_mean_ele.text
                    sleep(0.25)
                else: word_mean = ''
            except Exception as bug:
                word_mean = ''
                save_bug.update({'KUNYOMI': bug})
                pass
            print(">> Word mean: ", word_mean)

            # ================= WORD IMAGE HOLDER ====================
            try:
                word_image_holder_ele = driver.find_element(By.CSS_SELECTOR, f'div[id="image-holder-{word}"]')
                if word_image_holder_ele:   
                    word_image_svg_code = word_image_holder_ele.get_attribute('innerHTML')
                    print(f">> Image svg code: [ {word_image_svg_code[:50]} .... {word_image_svg_code[-50:]}]")
                    sleep(0.25)
            except Exception as bug:
                word_image_holder_ele = ''
                save_bug.update({'IMAGE HOLDER': bug})
                pass

            result_data = {
                'status': 'success',
                'word': word,
                'stroke': stroke_num,
                'vn_sound': vn_sound,
                'onyomi': onyomi,
                'kunyomi': kunyomi,
                'word_mean': word_mean,
                'word_image_svg_code':  word_image_svg_code
            }

            break
        except Exception as bug:
            save_bug.update({'try: for __ (3)': bug})
            result_data = {
                'status': 'failed', 
                'reason': save_bug, 
                'function': 'get_word_data(word)', 
                'value': word
            }
            continue

    return result_data

In [95]:
kanji_mazzi_domain = 'https://vi.mazii.net/vi-VN/search/kanji/'

database_folder = 'Kanji Database'
kanji_stroke_folder = 'Stroke Image backup'
craw_data_file_name = 'words_craw_update.json'
result_data_file_name = 'Course_mazzi backup.json'

kanji_words = json_read(os.path.join(database_folder, craw_data_file_name))

In [96]:
course = 'Course'
course_data = {}
for lesson, word_list in kanji_words[course].items():
    lesson_data = []
    for word in word_list:
        word_index = word_list.index(word) + 1
        print('\n' + '-' * 120)
        print(f'>> Lesson: [ {lesson} ] - Index: [ {word_index} ] - Word: [ {word} ]')
        word_data = get_word_data(driver, word)
        if word_data['status'] == 'success':
            stroke_img_file_name = f"{lesson}_{word_index}_{word}.png"
            stroke_img_save_folder = f"{database_folder}\\{kanji_stroke_folder}\\{lesson}"
            save_svg_image(
                code=word_data['word_image_svg_code'],
                img_name=stroke_img_file_name,
                path=stroke_img_save_folder
            )
            print(f'>> Save Stroke Image: [ Success ]')

            lesson_data.append({
                'word_index': word_index,
                **word_data
            })  
        else:
            lesson_data.append({
                'word_index': word_index,
                'lesson': lesson,
                'status': 'failed'
            })  

        course_data.update({
            f"{lesson}": lesson_data,
        })

    json_write(f'{database_folder}\\{result_data_file_name}', {f"{course}": course_data})


------------------------------------------------------------------------------------------------------------------------
>> Lesson: [ 29 ] - Index: [ 6 ] - Word: [ 曜 ]
>> Stroke:  18
>> Wait draw time: [==         ] [ OKE ]
>> Vn sound:  
>> Onyomi:  ヨウ
>> Kunyomi:  
>> Word mean:  Nghĩa: Bóng sáng mặt trời. Mặt trời, mặt trăng, sao đều gọi là diệu.
Giải nghĩa:
Bóng sáng mặt trời.
>> Image svg code: [ <svg height="250px" version="1.1" width="250px" xm .... -color: rgba(0, 0, 0, 0);">18</tspan></text></svg>]
>> Save Stroke Image: [ Success ]


In [97]:
driver.quit()

In [98]:
course_mazzi_data = json_read(os.path.join(database_folder, result_data_file_name))
course = 'Course'
course_data = {}
for lesson, word_list in course_mazzi_data[course].items():
    lesson_data = []
    for word in word_list:
        if word['status'] == 'failed':
            print(f'>> Lesson: [ {lesson} ] - Word: [ {word} ]')
            word_data = get_word_data(driver, word['word'])
            if word_data['status'] == 'success':
                stroke_img_file_name = f"{word['lesson']}_{word['word_index']}_{word['word']}.png"
                stroke_img_save_folder = f"{database_folder}\\{kanji_stroke_folder}\\{word['lesson']}"
                save_svg_image(
                    code=word_data['word_image_svg_code'],
                    img_name=stroke_img_file_name,
                    path=stroke_img_save_folder
                )
                print(f'>> Save Stroke Image: [ Success ]')

                lesson_data.append({
                    'word_index': word['word_index'],
                    **word_data
                })  
            else:
                lesson_data.append({
                    'word_index': word['word_index'],
                    'lesson': lesson,
                    'status': 'failed'
                })  

            course_data.update({
                f"{lesson}": lesson_data,
            })

        json_write(f'{database_folder}\\{result_data_file_name}_failed.json', {f"{course}": course_data})